In [46]:
import numpy as np
import os
import pandas as pd

import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import seaborn as sns

import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras.layers import Dropout, InputLayer, Activation
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit

import warnings
warnings.filterwarnings(action='ignore')

#한글설정
import matplotlib.font_manager as fm

font_dirs = ['/usr/share/fonts/truetype/nanum', ]
font_files = fm.findSystemFonts(fontpaths=font_dirs)

for font_file in font_files:
    fm.fontManager.addfont(font_file)
    
# 한글 출력을 위해서 폰트 옵션을 설정합니다.
# "axes.unicode_minus" : 마이너스가 깨질 것을 방지

sns.set(font="NanumBarunGothic",
        rc={"axes.unicode_minus":False},
        style='darkgrid')

#GPU 사용 설정, -1이면 CPU 사용
# os.environ["CUDA_VISIBLE_DEVICES"] = "0, 1"

gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
if gpus:
    try:
        for i in range(len(gpus)):
            tf.config.experimental.set_memory_growth(gpus[i], True)
    except RuntimeError as e:
        # 프로그램 시작시에 메모리 증가가 설정되어야만 합니다
        print(e)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


# 시계열, LSTM

In [47]:
# 2. LSTM model
def lstm_model():
    seed_num = 42 # model_seed
    tf.random.set_seed(seed_num)

    lstm = Sequential()
    lstm.add(InputLayer(input_shape=(x.shape[1],x.shape[2])))
    lstm.add(LSTM(units=128, activation='hard_sigmoid', return_sequences=True))
    lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
    lstm.add(Dropout(0.2))
    lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
    lstm.add(LSTM(units=32, activation='hard_sigmoid', return_sequences=False))
    lstm.add(Dropout(0.2))
    lstm.add(Dense(units=1, activation='sigmoid'))
    return lstm

def sorted_ls(path):
    mtime=lambda f: os.stat(os.path.join(path, f)).st_mtime
    return list(sorted(os.listdir(path), key=mtime))

path = '/project/LSH/'
with tf.device('/device:GPU:0'):
    # 1. Data load
    x = np.load(path + 'x_(7727,10,4068).npy')
    y = np.load(path + 'y_(7727,1).npy')

    # 3-1. Best model saving
    MODEL_SAVE_FOLDER_PATH = './models/'
    if not os.path.exists(MODEL_SAVE_FOLDER_PATH):
        os.mkdir(MODEL_SAVE_FOLDER_PATH)

    data={}
    sss = StratifiedShuffleSplit(n_splits=10, test_size = 0.2, random_state = 42)

    # 4. Crossvalidation
    for seed, (train_index, test_index) in tqdm(enumerate(sss.split(x, y))):
        X_train, y_train = x[train_index,:,:], y[train_index]
        X_test, y_test = x[test_index,:,:], y[test_index]

        # 3-2. Best model saving
        model_path = MODEL_SAVE_FOLDER_PATH + f'/seed={seed}_'+'{epoch:02d}'+'_val_acc:{val_acc:.4f}.hdf5'

        cb_checkpoint = ModelCheckpoint(filepath=model_path, monitor='val_acc',
                                        verbose=1, save_best_only=True)
        early_stop = EarlyStopping(monitor='val_acc', patience=50, verbose=1, restore_best_weights=False)


        model = lstm_model()
        model.compile(optimizer= keras.optimizers.Adam(learning_rate = 0.001), 
                      loss = "binary_crossentropy", metrics=['acc'])
        model.fit(X_train, y_train, validation_split=0.25, batch_size=128, epochs=500,
                  callbacks=[early_stop, cb_checkpoint], shuffle=False)

        # 5. accuracy checking 
#         from pathlib import Path
#         paths = sorted(Path(MODEL_SAVE_FOLDER_PATH).iterdir(), key=os.path.getmtime)[-1]
#         best_model_path = str(paths, 'utf-8')
        best_model_path = MODEL_SAVE_FOLDER_PATH + sorted_ls('models/')[-1]
        print('############# model path : ', best_model_path)

        best_model = keras.models.load_model(best_model_path) 
#         y_pred_train = best_model.predict(X_train, batch_size=10000, workers=-1, use_multiprocessing=True)
        y_pred_test = best_model.predict(X_test, batch_size=10000, workers=-1, use_multiprocessing=True)

        y_pred_test[y_pred_test>0.5]=1
        y_pred_test[y_pred_test<=0.5]=0

        acc = accuracy_score(y_test, y_pred_test)
        data[seed]=acc
        print(f'정확도 : {acc}, seed_num = {seed}')

    df = pd.DataFrame.from_dict(data, orient='index')
    print(f'정확도 df 만들고 평균 확인 : {df.mean().values}')    

0it [00:00, ?it/s]

Epoch 1/500
37/37 [==============================] - 6s 79ms/step - loss: 0.6753 - acc: 0.6004 - val_loss: 0.6704 - val_acc: 0.6054

Epoch 00001: val_acc improved from -inf to 0.60543, saving model to ./models/seed=0_01_val_acc:0.6054.hdf5
Epoch 2/500
37/37 [==============================] - 3s 71ms/step - loss: 0.6696 - acc: 0.6097 - val_loss: 0.6541 - val_acc: 0.6054

Epoch 00002: val_acc did not improve from 0.60543
Epoch 3/500
37/37 [==============================] - 3s 69ms/step - loss: 0.5884 - acc: 0.6785 - val_loss: 0.5440 - val_acc: 0.7296

Epoch 00003: val_acc improved from 0.60543 to 0.72962, saving model to ./models/seed=0_03_val_acc:0.7296.hdf5
Epoch 4/500
37/37 [==============================] - 3s 71ms/step - loss: 0.4601 - acc: 0.7970 - val_loss: 0.5489 - val_acc: 0.7393

Epoch 00004: val_acc improved from 0.72962 to 0.73933, saving model to ./models/seed=0_04_val_acc:0.7393.hdf5
Epoch 5/500
37/37 [==============================] - 3s 71ms/step - loss: 0.4108 - acc: 0.8

37/37 [==============================] - 3s 70ms/step - loss: 0.1294 - acc: 0.9577 - val_loss: 1.0653 - val_acc: 0.7180

Epoch 00041: val_acc did not improve from 0.74515
Epoch 42/500
37/37 [==============================] - 3s 70ms/step - loss: 0.1313 - acc: 0.9547 - val_loss: 1.0884 - val_acc: 0.6966

Epoch 00042: val_acc did not improve from 0.74515
Epoch 43/500
37/37 [==============================] - 3s 70ms/step - loss: 0.1265 - acc: 0.9594 - val_loss: 0.9660 - val_acc: 0.7316

Epoch 00043: val_acc did not improve from 0.74515
Epoch 44/500
37/37 [==============================] - 3s 70ms/step - loss: 0.0688 - acc: 0.9847 - val_loss: 1.0030 - val_acc: 0.7277

Epoch 00044: val_acc did not improve from 0.74515
Epoch 45/500
37/37 [==============================] - 3s 69ms/step - loss: 0.0608 - acc: 0.9853 - val_loss: 1.0478 - val_acc: 0.7257

Epoch 00045: val_acc did not improve from 0.74515
Epoch 46/500
37/37 [==============================] - 3s 70ms/step - loss: 0.0613 - acc: 0.98


Epoch 00014: val_acc did not improve from 0.77684
Epoch 15/500
37/37 [==============================] - 3s 70ms/step - loss: 0.2279 - acc: 0.9184 - val_loss: 0.6949 - val_acc: 0.7309

Epoch 00015: val_acc did not improve from 0.77684
Epoch 16/500
37/37 [==============================] - 3s 69ms/step - loss: 0.2172 - acc: 0.9243 - val_loss: 0.6914 - val_acc: 0.7400

Epoch 00016: val_acc did not improve from 0.77684
Epoch 17/500
37/37 [==============================] - 3s 69ms/step - loss: 0.2327 - acc: 0.9118 - val_loss: 0.6651 - val_acc: 0.7555

Epoch 00017: val_acc did not improve from 0.77684
Epoch 18/500
37/37 [==============================] - 3s 69ms/step - loss: 0.2292 - acc: 0.9120 - val_loss: 0.6595 - val_acc: 0.7549

Epoch 00018: val_acc did not improve from 0.77684
Epoch 19/500
37/37 [==============================] - 3s 70ms/step - loss: 0.1655 - acc: 0.9465 - val_loss: 0.7198 - val_acc: 0.7471

Epoch 00019: val_acc did not improve from 0.77684
Epoch 20/500
37/37 [=========

정확도 : 0.7729624838292367, seed_num = 1
Epoch 1/500
37/37 [==============================] - 6s 88ms/step - loss: 0.6766 - acc: 0.6011 - val_loss: 0.6684 - val_acc: 0.6171

Epoch 00001: val_acc improved from -inf to 0.61708, saving model to ./models/seed=2_01_val_acc:0.6171.hdf5
Epoch 2/500
37/37 [==============================] - 3s 70ms/step - loss: 0.6718 - acc: 0.6011 - val_loss: 0.6474 - val_acc: 0.6171

Epoch 00002: val_acc did not improve from 0.61708
Epoch 3/500
37/37 [==============================] - 3s 69ms/step - loss: 0.5979 - acc: 0.6783 - val_loss: 0.5098 - val_acc: 0.7671

Epoch 00003: val_acc improved from 0.61708 to 0.76714, saving model to ./models/seed=2_03_val_acc:0.7671.hdf5
Epoch 4/500
37/37 [==============================] - 3s 69ms/step - loss: 0.4723 - acc: 0.7883 - val_loss: 0.4718 - val_acc: 0.7820

Epoch 00004: val_acc improved from 0.76714 to 0.78202, saving model to ./models/seed=2_04_val_acc:0.7820.hdf5
Epoch 5/500
37/37 [==============================] -

...

37/37 [==============================] - 3s 72ms/step - loss: 0.1483 - acc: 0.9558 - val_loss: 0.7899 - val_acc: 0.7542

Epoch 00022: val_acc did not improve from 0.76585
Epoch 23/500
37/37 [==============================] - 3s 74ms/step - loss: 0.1360 - acc: 0.9631 - val_loss: 0.8553 - val_acc: 0.7477

Epoch 00023: val_acc did not improve from 0.76585
Epoch 24/500
37/37 [==============================] - 3s 74ms/step - loss: 0.1295 - acc: 0.9640 - val_loss: 0.9293 - val_acc: 0.7439

Epoch 00024: val_acc did not improve from 0.76585
Epoch 25/500
37/37 [==============================] - 3s 73ms/step - loss: 0.1330 - acc: 0.9650 - val_loss: 0.9640 - val_acc: 0.7432

Epoch 00025: val_acc did not improve from 0.76585
Epoch 26/500
37/37 [==============================] - 3s 73ms/step - loss: 0.1377 - acc: 0.9618 - val_loss: 0.9747 - val_acc: 0.7439

Epoch 00026: val_acc did not improve from 0.76585
Epoch 27/500
37/37 [==============================] - 3s 74ms/step - loss: 0.1425 - acc: 0.96

37/37 [==============================] - 3s 72ms/step - loss: 0.0592 - acc: 0.9879 - val_loss: 1.1245 - val_acc: 0.7419

Epoch 00066: val_acc did not improve from 0.76843
Epoch 67/500
37/37 [==============================] - 3s 73ms/step - loss: 0.0574 - acc: 0.9888 - val_loss: 1.1559 - val_acc: 0.7335

Epoch 00067: val_acc did not improve from 0.76843
Epoch 68/500
37/37 [==============================] - 3s 72ms/step - loss: 0.0646 - acc: 0.9862 - val_loss: 1.1068 - val_acc: 0.7413

Epoch 00068: val_acc did not improve from 0.76843
Epoch 69/500
37/37 [==============================] - 3s 73ms/step - loss: 0.0599 - acc: 0.9879 - val_loss: 1.1091 - val_acc: 0.7335

Epoch 00069: val_acc did not improve from 0.76843
Epoch 70/500
37/37 [==============================] - 3s 74ms/step - loss: 0.0651 - acc: 0.9851 - val_loss: 1.1314 - val_acc: 0.7303

Epoch 00070: val_acc did not improve from 0.76843
Epoch 71/500
37/37 [==============================] - 3s 74ms/step - loss: 0.0589 - acc: 0.98

37/37 [==============================] - 3s 74ms/step - loss: 0.1260 - acc: 0.9642 - val_loss: 0.8748 - val_acc: 0.7361

Epoch 00020: val_acc did not improve from 0.76714
Epoch 21/500
37/37 [==============================] - 3s 75ms/step - loss: 0.1616 - acc: 0.9456 - val_loss: 0.8272 - val_acc: 0.7380

Epoch 00021: val_acc did not improve from 0.76714
Epoch 22/500
37/37 [==============================] - 3s 75ms/step - loss: 0.1994 - acc: 0.9286 - val_loss: 0.8338 - val_acc: 0.7277

Epoch 00022: val_acc did not improve from 0.76714
Epoch 23/500
37/37 [==============================] - 3s 76ms/step - loss: 0.2247 - acc: 0.9131 - val_loss: 0.6956 - val_acc: 0.7458

Epoch 00023: val_acc did not improve from 0.76714
Epoch 24/500
37/37 [==============================] - 3s 74ms/step - loss: 0.1265 - acc: 0.9644 - val_loss: 0.8032 - val_acc: 0.7303

Epoch 00024: val_acc did not improve from 0.76714
Epoch 25/500
37/37 [==============================] - 3s 74ms/step - loss: 0.1141 - acc: 0.96

37/37 [==============================] - 3s 74ms/step - loss: 0.5530 - acc: 0.7161 - val_loss: 0.5426 - val_acc: 0.7354

Epoch 00003: val_acc improved from 0.58991 to 0.73545, saving model to ./models/seed=9_03_val_acc:0.7354.hdf5
Epoch 4/500
37/37 [==============================] - 3s 73ms/step - loss: 0.4424 - acc: 0.8078 - val_loss: 0.5434 - val_acc: 0.7490

Epoch 00004: val_acc improved from 0.73545 to 0.74903, saving model to ./models/seed=9_04_val_acc:0.7490.hdf5
Epoch 5/500
37/37 [==============================] - 3s 75ms/step - loss: 0.3838 - acc: 0.8403 - val_loss: 0.6018 - val_acc: 0.7348

Epoch 00005: val_acc did not improve from 0.74903
Epoch 6/500
37/37 [==============================] - 3s 73ms/step - loss: 0.3586 - acc: 0.8520 - val_loss: 0.5732 - val_acc: 0.7542

Epoch 00006: val_acc improved from 0.74903 to 0.75420, saving model to ./models/seed=9_06_val_acc:0.7542.hdf5
Epoch 7/500
37/37 [==============================] - 3s 75ms/step - loss: 0.3308 - acc: 0.8652 - val

37/37 [==============================] - 3s 75ms/step - loss: 0.0554 - acc: 0.9875 - val_loss: 1.0810 - val_acc: 0.7232

Epoch 00046: val_acc did not improve from 0.76391
Epoch 47/500
37/37 [==============================] - 3s 75ms/step - loss: 0.0545 - acc: 0.9881 - val_loss: 1.1162 - val_acc: 0.7212

Epoch 00047: val_acc did not improve from 0.76391
Epoch 48/500
37/37 [==============================] - 3s 74ms/step - loss: 0.0504 - acc: 0.9905 - val_loss: 1.1321 - val_acc: 0.7225

Epoch 00048: val_acc did not improve from 0.76391
Epoch 49/500
37/37 [==============================] - 3s 73ms/step - loss: 0.0503 - acc: 0.9888 - val_loss: 1.1498 - val_acc: 0.7206

Epoch 00049: val_acc did not improve from 0.76391
Epoch 50/500
37/37 [==============================] - 3s 74ms/step - loss: 0.0505 - acc: 0.9890 - val_loss: 1.1593 - val_acc: 0.7212

Epoch 00050: val_acc did not improve from 0.76391
Epoch 51/500
37/37 [==============================] - 3s 75ms/step - loss: 0.0479 - acc: 0.98

- 정확도 df 만들고 평균 확인 : [0.75789133]

In [48]:
df

,0
0,0.758085
1,0.772962
2,0.756792
3,0.751617
4,0.774903
5,0.774256
6,0.773609
7,0.733506
8,0.760673
9,0.722510


# 비시계열

In [49]:
seed_num = 46
import random
random.seed(seed_num) # data shuffle할 때의 random seed 

x = np.load(path + 'x_(7727,4068).npy')
y = np.load(path + 'y_(7727,1).npy')

idx = list(range(len(x)))
random.shuffle(idx)

i = round(x.shape[0]*0.8)
X_train, y_train = x[idx[:i],:], y[idx[:i]]
X_test, y_test = x[idx[i:],:], y[idx[i:]]

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((6182, 4068), (1545, 4068), (6182,), (1545,))

In [50]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Support vector machine
model = SVC()
model.fit(X_train, y_train)
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)
print("Support vector machine:", accuracy_score(y_train, y_pred_train), accuracy_score(y_test, y_pred_test))

# Gaussian naive bayes
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(X_train, y_train)
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)
print("Gaussian naive bayes:", accuracy_score(y_train, y_pred_train), accuracy_score(y_test, y_pred_test))

# K nearest neighbor
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier()
model.fit(X_train, y_train)
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)
print("K nearest neighbor:", accuracy_score(y_train, y_pred_train), accuracy_score(y_test, y_pred_test))

# Random Forest
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(X_train, y_train)
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)
print("Random forest:", accuracy_score(y_train, y_pred_train), accuracy_score(y_test, y_pred_test))

# Gradient boosting
from sklearn.ensemble import GradientBoostingClassifier
model = GradientBoostingClassifier()
model.fit(X_train, y_train)
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)
print("Gradient boosing:", accuracy_score(y_train, y_pred_train), accuracy_score(y_test, y_pred_test))

# Neural network
from sklearn.neural_network import MLPClassifier
model = MLPClassifier(max_iter=1000)
model.fit(X_train, y_train)
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)
print("Neural network:", accuracy_score(y_train, y_pred_train), accuracy_score(y_test, y_pred_test))

# AdaBoost
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score

clf = AdaBoostClassifier(n_estimators=500, 
                        random_state=10, 
                        learning_rate=0.1)
clf.fit(X_train, y_train)
pred = clf.predict(X_test)
print('AdaBoost 정확도: {:.4f}'.format(accuracy_score(y_test, pred)))

Support vector machine: 0.9037528307990942 0.7650485436893204
Gaussian naive bayes: 0.5598511808476221 0.4854368932038835
K nearest neighbor: 0.7500808799741184 0.6608414239482201
Random forest: 1.0 0.7385113268608414
Gradient boosing: 0.7935943060498221 0.7372168284789644
Neural network: 1.0 0.7249190938511327
AdaBoost 정확도: 0.7482
